In [41]:
import pandas as pd

## Imports the data and explores it
Steps will be:
- Import the data
- Check out the data
- Check the info
- Describe the data
- Check the shape

In [65]:
# Imports the data
bom_raw = pd.read_csv("../data/bom.movie_gross.csv.gz")
rt_movie_info_raw = pd.read_csv("../data/rt.movie_info.tsv.gz", delimiter="\t")
rt_reviews_raw = pd.read_csv("../data/rt.reviews.tsv.gz", delimiter="\t", encoding='cp1252')
tmdb_movies_raw = pd.read_csv("../data/tmdb.movies.csv.gz")
tn_movie_budgets_raw = pd.read_csv("../data/tn.movie_budgets.csv.gz")

In [66]:
# rt_review_data_raw initially doesn't load due to an encoding error. Here we detect the encoding type before inserting it into the codeblock above
with open("../data/rt.reviews.tsv.gz") as rt_reviews:
    print(rt_reviews)

<_io.TextIOWrapper name='../data/rt.reviews.tsv.gz' mode='r' encoding='cp1252'>


In [73]:
# Check out the data
bom_raw.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [74]:
rt_movie_info_raw.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [69]:
rt_reviews_raw.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [70]:
tmdb_movies_raw.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [71]:
tn_movie_budgets_raw.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


**What does a row represent?**
- A

In [51]:
# Check the information of the data set
bom_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [52]:
rt_movie_info_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


In [53]:
rt_reviews_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          54432 non-null  int64 
 1   review      48869 non-null  object
 2   rating      40915 non-null  object
 3   fresh       54432 non-null  object
 4   critic      51710 non-null  object
 5   top_critic  54432 non-null  int64 
 6   publisher   54123 non-null  object
 7   date        54432 non-null  object
dtypes: int64(2), object(6)
memory usage: 3.3+ MB


In [75]:
tmdb_movies_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB


In [76]:
tn_movie_budgets_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


**Observations**
- It looks like we have mostly strings, with 5 numbers at the moment
- We will have to handle dates
- There are a significant number of nulls

In [5]:
# Describe the numerical data
aviation_data_raw.describe()

,Number.of.Engines,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
count,82805.000000,77488.000000,76379.000000,76956.000000,82977.000000
mean,1.146585,0.647855,0.279881,0.357061,5.325440
std,0.446510,5.485960,1.544084,2.235625,27.913634
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,0.000000,0.000000,0.000000,2.000000
max,8.000000,349.000000,161.000000,380.000000,699.000000


In [6]:
# Describle the non-numerical data
aviation_data_raw.describe(include=object)

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Amateur.Built,Engine.Type,FAR.Description,Schedule,Purpose.of.flight,Air.carrier,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
count,88889,90348,88889,88889,88837,88663,34382,34373,50249,52790,...,88787,81812,32023,12582,82697,16648,84397,61724,82508,73659
unique,87951,71,88863,14782,27758,219,25592,27156,10375,24871,...,2,13,31,3,26,13590,4,12,17007,2923
top,20001214X45071,Accident,GAA22WA241,1984-06-30,"ANCHORAGE, AK",United States,332739N,0112457W,NONE,Private,...,No,Reciprocating,091,NSCH,Personal,Pilot,VMC,Landing,Probable Cause,25-09-2020
freq,3,85015,2,25,434,82248,19,24,1488,240,...,80312,69530,18221,4474,49448,258,77303,15428,61754,16317


In [7]:
# Check the shape
aviation_data_raw.shape

(90348, 31)

In [8]:
# Create a copy for cleaning
aviation_data_cleaned = aviation_data_raw

## Handling Column Names
We will start by cleaning the column names so that they will be easy to access
We will:
- Strip spaces
- Replace periods with underscores
- lowercase all column names

In [9]:
# Check out column names
aviation_data_cleaned.columns

Index(['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date',
       'Location', 'Country', 'Latitude', 'Longitude', 'Airport.Code',
       'Airport.Name', 'Injury.Severity', 'Aircraft.damage',
       'Aircraft.Category', 'Registration.Number', 'Make', 'Model',
       'Amateur.Built', 'Number.of.Engines', 'Engine.Type', 'FAR.Description',
       'Schedule', 'Purpose.of.flight', 'Air.carrier', 'Total.Fatal.Injuries',
       'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured',
       'Weather.Condition', 'Broad.phase.of.flight', 'Report.Status',
       'Publication.Date'],
      dtype='object')

In [10]:
# Start by stripping white space
aviation_data_cleaned.columns = aviation_data_cleaned.columns.str.strip()
aviation_data_cleaned.columns

Index(['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date',
       'Location', 'Country', 'Latitude', 'Longitude', 'Airport.Code',
       'Airport.Name', 'Injury.Severity', 'Aircraft.damage',
       'Aircraft.Category', 'Registration.Number', 'Make', 'Model',
       'Amateur.Built', 'Number.of.Engines', 'Engine.Type', 'FAR.Description',
       'Schedule', 'Purpose.of.flight', 'Air.carrier', 'Total.Fatal.Injuries',
       'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured',
       'Weather.Condition', 'Broad.phase.of.flight', 'Report.Status',
       'Publication.Date'],
      dtype='object')

In [11]:
# Replace . with _
aviation_data_cleaned.columns = aviation_data_cleaned.columns.str.replace(".","_")
aviation_data_cleaned.columns

Index(['Event_Id', 'Investigation_Type', 'Accident_Number', 'Event_Date',
       'Location', 'Country', 'Latitude', 'Longitude', 'Airport_Code',
       'Airport_Name', 'Injury_Severity', 'Aircraft_damage',
       'Aircraft_Category', 'Registration_Number', 'Make', 'Model',
       'Amateur_Built', 'Number_of_Engines', 'Engine_Type', 'FAR_Description',
       'Schedule', 'Purpose_of_flight', 'Air_carrier', 'Total_Fatal_Injuries',
       'Total_Serious_Injuries', 'Total_Minor_Injuries', 'Total_Uninjured',
       'Weather_Condition', 'Broad_phase_of_flight', 'Report_Status',
       'Publication_Date'],
      dtype='object')

In [12]:
# Lowercase all names
aviation_data_cleaned.columns = aviation_data_cleaned.columns.str.lower()
aviation_data_cleaned.columns

Index(['event_id', 'investigation_type', 'accident_number', 'event_date',
       'location', 'country', 'latitude', 'longitude', 'airport_code',
       'airport_name', 'injury_severity', 'aircraft_damage',
       'aircraft_category', 'registration_number', 'make', 'model',
       'amateur_built', 'number_of_engines', 'engine_type', 'far_description',
       'schedule', 'purpose_of_flight', 'air_carrier', 'total_fatal_injuries',
       'total_serious_injuries', 'total_minor_injuries', 'total_uninjured',
       'weather_condition', 'broad_phase_of_flight', 'report_status',
       'publication_date'],
      dtype='object')

## Handling Duplicate
We will check for duplicates and see how we should handle them

In [13]:
#Check for duplicates
aviation_data_cleaned.duplicated().value_counts()

False    88958
True      1390
dtype: int64

In [14]:
# Investigated duplicates
aviation_data_cleaned[aviation_data_cleaned.duplicated(keep=False)]

,event_id,investigation_type,accident_number,event_date,location,country,latitude,longitude,airport_code,airport_name,...,purpose_of_flight,air_carrier,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight,report_status,publication_date
64030,NaN,25-09-2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64050,NaN,25-09-2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64052,NaN,25-09-2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64388,NaN,25-09-2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64541,NaN,25-09-2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90004,NaN,15-12-2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90010,NaN,15-12-2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90031,NaN,15-12-2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90090,NaN,20-12-2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The duplicates seem to be mostly empty data, we can drop these rows

In [15]:
# Drop the duplicates
aviation_data_cleaned = aviation_data_cleaned.drop_duplicates()

In [16]:
#Confirm there are no duplicates in the cleaned data
aviation_data_cleaned.duplicated().value_counts()

False    88958
dtype: int64

In [17]:
# Check out dataframe to see what other columns we may want to check for duplicates
aviation_data_cleaned.head()

,event_id,investigation_type,accident_number,event_date,location,country,latitude,longitude,airport_code,airport_name,...,purpose_of_flight,air_carrier,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight,report_status,publication_date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.9222,-81.8781,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


In [18]:
# Check for duplicates in event_id
aviation_data_cleaned['event_id'].duplicated().value_counts()

False    87952
True      1006
Name: event_id, dtype: int64

In [19]:
# Investigate the event_id duplicates
aviation_data_cleaned[aviation_data_cleaned['event_id'].duplicated(keep=False)].sort_values(by=['event_id'])

,event_id,investigation_type,accident_number,event_date,location,country,latitude,longitude,airport_code,airport_name,...,purpose_of_flight,air_carrier,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight,report_status,publication_date
45562,20001204X00086,Accident,LAX99LA086B,1999-01-29,"MADERA, CA",United States,NaN,NaN,MAE,MADERA MUNICIPAL AIRPORT,...,Personal,NaN,0.0,0.0,0.0,2.0,VMC,Taxi,Probable Cause,30-11-2000
45564,20001204X00086,Accident,LAX99LA086A,1999-01-29,"MADERA, CA",United States,NaN,NaN,MAE,MADERA MUNICIPAL AIRPORT,...,Personal,NaN,0.0,0.0,0.0,2.0,VMC,Taxi,Probable Cause,30-11-2000
45703,20001205X00276,Incident,CHI99IA100B,1999-03-02,"SALINA, KS",United States,NaN,NaN,NaN,NaN,...,Unknown,NaN,0.0,0.0,0.0,6.0,VMC,Cruise,Probable Cause,30-11-2000
45704,20001205X00276,Incident,CHI99IA100A,1999-03-02,"SALINA, KS",United States,NaN,NaN,NaN,NaN,...,Unknown,NaN,0.0,0.0,0.0,6.0,VMC,Cruise,Probable Cause,30-11-2000
45716,20001205X00305,Accident,DEN99LA047B,1999-03-05,"DENVER, CO",United States,NaN,NaN,DEN,DENVER INTERNATIONAL,...,Unknown,NaN,0.0,0.0,0.0,2.0,VMC,Taxi,Probable Cause,25-04-2001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89440,NaN,02-12-2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89492,NaN,05-08-2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89763,NaN,08-12-2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89943,NaN,15-12-2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Observations**
- The duplicates seem to be caused by multiple accident numbers referring to the same event
- Some have different air carriers
- Some have different broad_phase_of_flight	
- We can likely keep one pair for each duplicate as the data is still good

In [20]:
# Drip duplicate events but keep the first reported event
aviation_data_cleaned = aviation_data_cleaned.drop_duplicates(subset=['event_id'], keep='first')

In [21]:
# Confirm there are no more event_id duplicates
aviation_data_cleaned['event_id'].duplicated().value_counts()

False    87952
Name: event_id, dtype: int64

### Removing undesired data 
We don't want all the data so let's remove what we don't want
- We only want USA based planes
- We don't want amateur built planes
- We don't want old data

In [22]:
# Investigate the data
aviation_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87952 entries, 0 to 90347
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   event_id                87951 non-null  object 
 1   investigation_type      87952 non-null  object 
 2   accident_number         87951 non-null  object 
 3   event_date              87951 non-null  object 
 4   location                87899 non-null  object 
 5   country                 87729 non-null  object 
 6   latitude                34212 non-null  object 
 7   longitude               34203 non-null  object 
 8   airport_code            49601 non-null  object 
 9   airport_name            52117 non-null  object 
 10  injury_severity         86961 non-null  object 
 11  aircraft_damage         84848 non-null  object 
 12  aircraft_category       32181 non-null  object 
 13  registration_number     86666 non-null  object 
 14  make                    87888 non-null

In [23]:
# Including data only from the USA
usa_filter = aviation_data_cleaned['country'] == 'United States'
aviation_data_cleaned = aviation_data_cleaned[usa_filter]

In [24]:
# Check the value counts of country
aviation_data_cleaned['country'].value_counts()

United States    81355
Name: country, dtype: int64

In [25]:
# Drop the column since there is now only one value
aviation_data_cleaned = aviation_data_cleaned.drop('country', axis=1)
aviation_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81355 entries, 0 to 90347
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   event_id                81355 non-null  object 
 1   investigation_type      81355 non-null  object 
 2   accident_number         81355 non-null  object 
 3   event_date              81355 non-null  object 
 4   location                81344 non-null  object 
 5   latitude                32103 non-null  object 
 6   longitude               32093 non-null  object 
 7   airport_code            48548 non-null  object 
 8   airport_name            50990 non-null  object 
 9   injury_severity         81250 non-null  object 
 10  aircraft_damage         79453 non-null  object 
 11  aircraft_category       28062 non-null  object 
 12  registration_number     81318 non-null  object 
 13  make                    81334 non-null  object 
 14  model                   81317 non-null

**Removing Amateur Built Airplanes**

In [26]:
# Check out the data for amateur_built column
aviation_data_cleaned['amateur_built'].value_counts()

No     73068
Yes     8267
Name: amateur_built, dtype: int64

In [27]:
# We don't want any amateur built airplanes so let's remove the data 
pro_filter = aviation_data_cleaned['amateur_built'] == 'No'
aviation_data_cleaned = aviation_data_cleaned[pro_filter]

In [28]:
# Now drop the amateur_built column since all remaining aircraft are professionally built
aviation_data_cleaned = aviation_data_cleaned.drop('amateur_built', axis=1)
aviation_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73068 entries, 0 to 90347
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   event_id                73068 non-null  object 
 1   investigation_type      73068 non-null  object 
 2   accident_number         73068 non-null  object 
 3   event_date              73068 non-null  object 
 4   location                73058 non-null  object 
 5   latitude                27974 non-null  object 
 6   longitude               27967 non-null  object 
 7   airport_code            43439 non-null  object 
 8   airport_name            45668 non-null  object 
 9   injury_severity         72963 non-null  object 
 10  aircraft_damage         71187 non-null  object 
 11  aircraft_category       24609 non-null  object 
 12  registration_number     73031 non-null  object 
 13  make                    73059 non-null  object 
 14  model                   73053 non-null

**Removing Old Data**

The dates are in the `Event.Date` column in the format 'YYYY-MM-DD'

In [29]:
# Check chaging the Event.Date column to dates
pd.to_datetime(aviation_data_cleaned['event_date']).head()

0   1948-10-24
1   1962-07-19
2   1974-08-30
3   1977-06-19
4   1979-08-02
Name: event_date, dtype: datetime64[ns]

In [30]:
# Update the column to the correct datetime type
aviation_data_cleaned['event_date'] = pd.to_datetime(aviation_data_cleaned['event_date'])

In [31]:
# Confirm that it worked
aviation_data_cleaned['event_date'].head()

0   1948-10-24
1   1962-07-19
2   1974-08-30
3   1977-06-19
4   1979-08-02
Name: event_date, dtype: datetime64[ns]

In [32]:
aviation_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73068 entries, 0 to 90347
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   event_id                73068 non-null  object        
 1   investigation_type      73068 non-null  object        
 2   accident_number         73068 non-null  object        
 3   event_date              73068 non-null  datetime64[ns]
 4   location                73058 non-null  object        
 5   latitude                27974 non-null  object        
 6   longitude               27967 non-null  object        
 7   airport_code            43439 non-null  object        
 8   airport_name            45668 non-null  object        
 9   injury_severity         72963 non-null  object        
 10  aircraft_damage         71187 non-null  object        
 11  aircraft_category       24609 non-null  object        
 12  registration_number     73031 non-null  object

Great! It looks like it worked, now let's investigate the timframes we are working with

In [33]:
aviation_data_cleaned['event_date'].dt.year.value_counts()

1982    3275
1983    3196
1984    3143
1985    2842
1986    2597
1987    2565
1988    2452
1989    2283
1990    2243
1991    2178
1992    2027
1993    2016
1995    1965
1994    1957
1996    1871
1997    1816
1999    1813
1998    1807
2000    1765
2003    1694
2001    1653
2002    1629
2005    1597
2007    1566
2004    1565
2008    1485
2006    1427
2011    1382
2010    1364
2009    1363
2012    1356
2016    1166
2013    1162
2018    1154
2017    1142
2015    1130
2019    1128
2014    1119
2022    1114
2021    1067
2020     987
1979       2
1977       1
1948       1
1981       1
1962       1
1974       1
Name: event_date, dtype: int64

In [34]:
# There are only a few datapoints before 1082 so let's remove them
aviation_data_cleaned = aviation_data_cleaned[aviation_data_cleaned['event_date'].dt.year >= 1982]

In [35]:
# Check to make sure we now only have the dates we want
aviation_data_cleaned['event_date'].dt.year.value_counts()

1982    3275
1983    3196
1984    3143
1985    2842
1986    2597
1987    2565
1988    2452
1989    2283
1990    2243
1991    2178
1992    2027
1993    2016
1995    1965
1994    1957
1996    1871
1997    1816
1999    1813
1998    1807
2000    1765
2003    1694
2001    1653
2002    1629
2005    1597
2007    1566
2004    1565
2008    1485
2006    1427
2011    1382
2010    1364
2009    1363
2012    1356
2016    1166
2013    1162
2018    1154
2017    1142
2015    1130
2019    1128
2014    1119
2022    1114
2021    1067
2020     987
Name: event_date, dtype: int64

In [36]:
aviation_data_cleaned['publication_date'].value_counts()

25-09-2020    12704
26-09-2020     1079
31-03-1993      399
25-11-2003      346
15-02-2001      325
              ...  
17-12-2020        1
19-09-2017        1
10-02-2009        1
01-08-2001        1
14-04-2005        1
Name: publication_date, Length: 1986, dtype: int64

### Investigate the type of investitaion

In [37]:
# Check the value counts
aviation_data_cleaned['investigation_type'].value_counts()

Accident    70848
Incident     2213
Name: investigation_type, dtype: int64

In [38]:
# Check out what incidents look like
aviation_data_cleaned[aviation_data_cleaned['investigation_type']=='Incident']['report_status'].value_counts()

Probable Cause                                                                                                                                                                                                                                                                                                                                                                                               1749
Foreign                                                                                                                                                                                                                                                                                                                                                                                                         4
A failure of the nose landing gear end cap due to fatigue. Contributing to the failure were the ineffective inspection and the unknown effect of grain direction on fatigue life.                   

In [39]:
# Remove the incidents, only keep accidents
aviation_data_cleaned = aviation_data_cleaned[aviation_data_cleaned['investigation_type']=='Accident']

## Removing unneeded columns
Columns we don't need include:
- event_id: we've made the dataset unique on event_id and we don't need the id for joins
- accident_number: we already have an event_id and have removed duplicates because of accident_numbers
- registration_number: private information not needed for the analysis
- publication_date: we don't need to know when the data was published for the analysis
- investigation_type: we already filtered to just accidents
- report_status: the data is long sentance form and we don't have the ability to parse it

In [40]:
aviation_data_cleaned = aviation_data_cleaned.drop([
    'event_id',
    'accident_number',
    'registration_number',
    'publication_date',
    'investigation_type',
    'report_status'
    ], axis=1)
aviation_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70848 entries, 7 to 90347
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   event_date              70848 non-null  datetime64[ns]
 1   location                70838 non-null  object        
 2   latitude                27253 non-null  object        
 3   longitude               27246 non-null  object        
 4   airport_code            41898 non-null  object        
 5   airport_name            44097 non-null  object        
 6   injury_severity         70798 non-null  object        
 7   aircraft_damage         69747 non-null  object        
 8   aircraft_category       24000 non-null  object        
 9   make                    70848 non-null  object        
 10  model                   70842 non-null  object        
 11  number_of_engines       69305 non-null  float64       
 12  engine_type             68237 non-null  object

We also want to remove columns with low data coverage

In [41]:
missing_data = (aviation_data_cleaned.isna().sum() / len(aviation_data_cleaned)).sort_values(ascending=False)
missing_data

schedule                  0.881817
air_carrier               0.827179
aircraft_category         0.661247
far_description           0.660470
longitude                 0.615430
latitude                  0.615331
airport_code              0.408621
airport_name              0.377583
broad_phase_of_flight     0.251609
total_serious_injuries    0.134753
total_fatal_injuries      0.127188
total_minor_injuries      0.126807
total_uninjured           0.056882
engine_type               0.036854
purpose_of_flight         0.026098
number_of_engines         0.021779
aircraft_damage           0.015540
weather_condition         0.007594
injury_severity           0.000706
location                  0.000141
model                     0.000085
make                      0.000000
event_date                0.000000
dtype: float64

**Observations**
- We can likely exclude the columns with significant missing data
- We should keep those relevant to our business questions such as `airport_code` and `airport_name`

In [42]:
# Pull out columns to drop
columns_to_drop = missing_data[missing_data.values > 0.6].index.to_list()
columns_to_drop

['schedule',
 'air_carrier',
 'aircraft_category',
 'far_description',
 'longitude',
 'latitude']

In [43]:
aviation_data_cleaned = aviation_data_cleaned.drop(columns_to_drop, axis=1)
aviation_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70848 entries, 7 to 90347
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   event_date              70848 non-null  datetime64[ns]
 1   location                70838 non-null  object        
 2   airport_code            41898 non-null  object        
 3   airport_name            44097 non-null  object        
 4   injury_severity         70798 non-null  object        
 5   aircraft_damage         69747 non-null  object        
 6   make                    70848 non-null  object        
 7   model                   70842 non-null  object        
 8   number_of_engines       69305 non-null  float64       
 9   engine_type             68237 non-null  object        
 10  purpose_of_flight       68999 non-null  object        
 11  total_fatal_injuries    61837 non-null  float64       
 12  total_serious_injuries  61301 non-null  float6

## Handling NaNs
- Look through each column and decide how to handle NaN values


In [44]:
# Check out the data again
aviation_data_cleaned.head()

,event_date,location,airport_code,airport_name,injury_severity,aircraft_damage,make,model,number_of_engines,engine_type,purpose_of_flight,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight
7,1982-01-01,"PULLMAN, WA",NaN,BLACKBURN AG STRIP,Non-Fatal,Substantial,Cessna,140,1.0,Reciprocating,Personal,0.0,0.0,0.0,2.0,VMC,Takeoff
8,1982-01-01,"EAST HANOVER, NJ",N58,HANOVER,Non-Fatal,Substantial,Cessna,401B,2.0,Reciprocating,Business,0.0,0.0,0.0,2.0,IMC,Landing
9,1982-01-01,"JACKSONVILLE, FL",JAX,JACKSONVILLE INTL,Non-Fatal,Substantial,North American,NAVION L-17B,1.0,Reciprocating,Personal,0.0,0.0,3.0,0.0,IMC,Cruise
10,1982-01-01,"HOBBS, NM",NaN,NaN,Non-Fatal,Substantial,Piper,PA-28-161,1.0,Reciprocating,Personal,0.0,0.0,0.0,1.0,VMC,Approach
11,1982-01-01,"TUSKEGEE, AL",NaN,TUSKEGEE,Non-Fatal,Substantial,Beech,V35B,1.0,Reciprocating,Personal,0.0,0.0,0.0,1.0,VMC,Landing


In [45]:
# Explore the number of nulls
aviation_data_cleaned.isna().sum()

event_date                    0
location                     10
airport_code              28950
airport_name              26751
injury_severity              50
aircraft_damage            1101
make                          0
model                         6
number_of_engines          1543
engine_type                2611
purpose_of_flight          1849
total_fatal_injuries       9011
total_serious_injuries     9547
total_minor_injuries       8984
total_uninjured            4030
weather_condition           538
broad_phase_of_flight     17826
dtype: int64

### location

In [46]:
# Check data
aviation_data_cleaned['location'].head()

7          PULLMAN, WA
8     EAST HANOVER, NJ
9     JACKSONVILLE, FL
10           HOBBS, NM
11        TUSKEGEE, AL
Name: location, dtype: object

There are only 10 events with missing location data which is a crucial piece of of information so we can drop them

In [47]:
# Check data
aviation_data_cleaned = aviation_data_cleaned[aviation_data_cleaned['location'].notna()]

In [48]:
# Check to confirm we dropped nulles
aviation_data_cleaned['location'].isna().sum()

0

### Airport Code

In [49]:
# Check data
aviation_data_cleaned[['airport_code', 'airport_name']].head()

,airport_code,airport_name
7,NaN,BLACKBURN AG STRIP
8,N58,HANOVER
9,JAX,JACKSONVILLE INTL
10,NaN,NaN
11,NaN,TUSKEGEE


We have 30k missing airport codes and names, so we can replace missing with unknown

In [50]:
# Replace missing airport data with unknowns
aviation_data_cleaned[['airport_code', 'airport_name']] = aviation_data_cleaned[['airport_code', 'airport_name']].fillna('Unknown')

In [51]:
# Check dataset to make sure NaNs were removed
aviation_data_cleaned[['airport_code', 'airport_name']].head()

,airport_code,airport_name
7,Unknown,BLACKBURN AG STRIP
8,N58,HANOVER
9,JAX,JACKSONVILLE INTL
10,Unknown,Unknown
11,Unknown,TUSKEGEE


### Handling injuries

In [52]:
# Check out the number of missing data for injury data
aviation_data_cleaned[[
    'injury_severity', 
    'total_fatal_injuries', 
    'total_serious_injuries',
    'total_minor_injuries',
    'total_uninjured'
]].isna().sum()

injury_severity             50
total_fatal_injuries      9008
total_serious_injuries    9542
total_minor_injuries      8978
total_uninjured           4027
dtype: int64

In [53]:
# Check to see if the missing data overlaps
aviation_data_cleaned[[
    'injury_severity', 
    'total_fatal_injuries', 
    'total_serious_injuries',
    'total_minor_injuries',
    'total_uninjured'
]].isna().sum(axis=1).sum()

31605

We have 30k rows with at least one injury column being missing. Thus we can't just drop them all.

The total columns are numerical so replacing the values will require investigation

**Injury Severity**

In [54]:
# We can drop these rows since there's only 15
aviation_data_cleaned.dropna(subset=['injury_severity'], inplace=True)

In [55]:
aviation_data_cleaned[[
    'injury_severity', 
    'total_fatal_injuries', 
    'total_serious_injuries',
    'total_minor_injuries',
    'total_uninjured'
]].isna().sum()

injury_severity              0
total_fatal_injuries      9008
total_serious_injuries    9542
total_minor_injuries      8978
total_uninjured           4027
dtype: int64

**Handling the Totals**

In [56]:
aviation_data_cleaned[[
    'total_fatal_injuries', 
    'total_serious_injuries',
    'total_minor_injuries',
    'total_uninjured'
]].describe()

,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured
count,61780.000000,61246.000000,61810.000000,66761.000000
mean,0.427031,0.261144,0.338893,2.879615
std,2.573864,1.153832,1.327737,17.011328
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,1.000000
75%,0.000000,0.000000,0.000000,2.000000
max,265.000000,137.000000,125.000000,699.000000


**Observations**
- We have significant outliers so we can't replace missing with means
- We want all the data, uninjured can be used to figure out the deadliness of the crash
- min, 25%, 50%, and 75% are 0 for all but uninjured

Conclusion: 
- Replace missing injury records with 0 since most seem to be 0
- Replace missing uninjured counts with the median since we don't know the number of passengers

In [57]:
# Replace total uninjured with median
total_uninjured_median = aviation_data_cleaned['total_uninjured'].median()
aviation_data_cleaned['total_uninjured'].fillna(value=total_uninjured_median, inplace=True)

In [58]:
# Assume missing injury data had 0 in column
aviation_data_cleaned['total_fatal_injuries'].fillna(value=0, inplace=True)
aviation_data_cleaned['total_serious_injuries'].fillna(value=0, inplace=True)
aviation_data_cleaned['total_minor_injuries'].fillna(value=0, inplace=True)

In [59]:
# Check to make sure NaNs were removed
aviation_data_cleaned[[
    'injury_severity', 
    'total_fatal_injuries', 
    'total_serious_injuries',
    'total_minor_injuries',
    'total_uninjured'
]].isna().sum()

injury_severity           0
total_fatal_injuries      0
total_serious_injuries    0
total_minor_injuries      0
total_uninjured           0
dtype: int64

In [60]:
# Explore the number of nulls
aviation_data_cleaned.isna().sum()

event_date                    0
location                      0
airport_code                  0
airport_name                  0
injury_severity               0
aircraft_damage            1082
make                          0
model                         6
number_of_engines          1521
engine_type                2579
purpose_of_flight          1814
total_fatal_injuries          0
total_serious_injuries        0
total_minor_injuries          0
total_uninjured               0
weather_condition           514
broad_phase_of_flight     17776
dtype: int64

### Replacing categorical data with unknown
There is some categorical data we will want to replace with unknown:
- aircraft_damage
- make
- model
- purpose_of_flight
- weather_condition
- broad_phase_of_flight

In [61]:
# Replace NaNs in these columns with unknowns
aviation_data_cleaned['aircraft_damage'].fillna(value='Unknown', inplace=True)
aviation_data_cleaned['make'].fillna(value='Unknown', inplace=True)
aviation_data_cleaned['model'].fillna(value='Unknown', inplace=True)
aviation_data_cleaned['purpose_of_flight'].fillna(value='Unknown', inplace=True)
aviation_data_cleaned['weather_condition'].fillna(value='Unknown', inplace=True)
aviation_data_cleaned['broad_phase_of_flight'].fillna(value='Unknown', inplace=True)

In [62]:
# Explore the number of nulls
aviation_data_cleaned.isna().sum()

event_date                   0
location                     0
airport_code                 0
airport_name                 0
injury_severity              0
aircraft_damage              0
make                         0
model                        0
number_of_engines         1521
engine_type               2579
purpose_of_flight            0
total_fatal_injuries         0
total_serious_injuries       0
total_minor_injuries         0
total_uninjured              0
weather_condition            0
broad_phase_of_flight        0
dtype: int64

### Handle number of engines

In [63]:
aviation_data_cleaned[aviation_data_cleaned['number_of_engines'].isna()]

,event_date,location,airport_code,airport_name,injury_severity,aircraft_damage,make,model,number_of_engines,engine_type,purpose_of_flight,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight
3600,1983-01-01,"ARROYO GRANDE, CA",Unknown,Unknown,Non-Fatal,Unknown,Piccard,AX-6,NaN,Unknown,Personal,0.0,1.0,0.0,1.0,VMC,Landing
3741,1983-01-22,"NORTH SHORE, CA",Unknown,DESERT AIR SKY RANCH,Non-Fatal,Substantial,Schweizer,2-33A,NaN,Unknown,Instructional,0.0,1.0,0.0,0.0,VMC,Takeoff
3772,1983-01-29,"HEMPSTEAD, TX",35R,HEMPSTEAD,Fatal(1),Destroyed,Schweizer,SGS 1-26B,NaN,Unknown,Instructional,1.0,0.0,0.0,0.0,VMC,Approach
3870,1983-02-12,"LITTLEFIELD, TX",Q00,LITTLEFIELD,Non-Fatal,Substantial,Pratt-read,PRG-1,NaN,Unknown,Instructional,0.0,0.0,0.0,1.0,VMC,Cruise
4004,1983-02-28,"ANCHORAGE, AK",MRI,MERRILL FIELD,Non-Fatal,Unknown,Raven,RX-7,NaN,Unknown,Personal,0.0,0.0,0.0,5.0,VMC,Climb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90339,2022-12-18,"San Manual, AZ",Unknown,Unknown,Non-Fatal,Unknown,PIPER,PA28,NaN,NaN,Personal,0.0,0.0,0.0,3.0,Unknown,Unknown
90340,2022-12-21,"Auburn Hills, MI",Unknown,Unknown,Minor,Unknown,CESSNA,172F,NaN,NaN,Personal,0.0,1.0,0.0,0.0,Unknown,Unknown
90341,2022-12-21,"Reserve, LA",Unknown,Unknown,Minor,Unknown,GRUMMAN AMERICAN AVN. CORP.,AA-5B,NaN,NaN,Instructional,0.0,1.0,0.0,1.0,Unknown,Unknown
90343,2022-12-26,"Annapolis, MD",Unknown,Unknown,Minor,Unknown,PIPER,PA-28-151,NaN,NaN,Personal,0.0,1.0,0.0,0.0,Unknown,Unknown


In [64]:
# Create a map for the number of engines, take the first mode if there's a tie
filter_nans = aviation_data_cleaned['number_of_engines'].notna()
enginer_number_map = dict(aviation_data_cleaned[filter_nans].groupby(['make', 'model'])['number_of_engines'].agg(lambda x : pd.Series.mode(x)[0]))
list(enginer_number_map.keys())[:5]

[('1200', 'G103'),
 ('177MF LLC', 'PITTS MODEL 12'),
 ('2021FX3 LLC', 'CCX-2000'),
 ('A. Schleicher GMBH & Co.', 'ASW 27-18'),
 ('AAA AIRCRAFT LLC', 'CCX-2000')]

In [65]:
# Apply this map to the numer of enginers column to replace missing values
aviation_data_cleaned['engine_number_map'] = tuple(zip(aviation_data_cleaned['make'], aviation_data_cleaned['model']))
aviation_data_cleaned['engine_number_map']

7                               (Cessna, 140)
8                              (Cessna, 401B)
9              (North American, NAVION L-17B)
10                         (Piper, PA-28-161)
11                              (Beech, V35B)
                         ...                 
90340                          (CESSNA, 172F)
90341    (GRUMMAN AMERICAN AVN. CORP., AA-5B)
90343                      (PIPER, PA-28-151)
90345     (AMERICAN CHAMPION AIRCRAFT, 8GCBC)
90347                      (PIPER, PA-24-260)
Name: engine_number_map, Length: 70788, dtype: object

In [66]:
# Map the engine number to the new column
aviation_data_cleaned['engine_number_map'] = aviation_data_cleaned['engine_number_map'].map(enginer_number_map)

In [67]:
# Check out our data with the new column
aviation_data_cleaned.head()

,event_date,location,airport_code,airport_name,injury_severity,aircraft_damage,make,model,number_of_engines,engine_type,purpose_of_flight,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight,engine_number_map
7,1982-01-01,"PULLMAN, WA",Unknown,BLACKBURN AG STRIP,Non-Fatal,Substantial,Cessna,140,1.0,Reciprocating,Personal,0.0,0.0,0.0,2.0,VMC,Takeoff,1.0
8,1982-01-01,"EAST HANOVER, NJ",N58,HANOVER,Non-Fatal,Substantial,Cessna,401B,2.0,Reciprocating,Business,0.0,0.0,0.0,2.0,IMC,Landing,2.0
9,1982-01-01,"JACKSONVILLE, FL",JAX,JACKSONVILLE INTL,Non-Fatal,Substantial,North American,NAVION L-17B,1.0,Reciprocating,Personal,0.0,0.0,3.0,0.0,IMC,Cruise,1.0
10,1982-01-01,"HOBBS, NM",Unknown,Unknown,Non-Fatal,Substantial,Piper,PA-28-161,1.0,Reciprocating,Personal,0.0,0.0,0.0,1.0,VMC,Approach,1.0
11,1982-01-01,"TUSKEGEE, AL",Unknown,TUSKEGEE,Non-Fatal,Substantial,Beech,V35B,1.0,Reciprocating,Personal,0.0,0.0,0.0,1.0,VMC,Landing,1.0


In [68]:
# Confirm the new column type
aviation_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70788 entries, 7 to 90347
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   event_date              70788 non-null  datetime64[ns]
 1   location                70788 non-null  object        
 2   airport_code            70788 non-null  object        
 3   airport_name            70788 non-null  object        
 4   injury_severity         70788 non-null  object        
 5   aircraft_damage         70788 non-null  object        
 6   make                    70788 non-null  object        
 7   model                   70788 non-null  object        
 8   number_of_engines       69267 non-null  float64       
 9   engine_type             68209 non-null  object        
 10  purpose_of_flight       70788 non-null  object        
 11  total_fatal_injuries    70788 non-null  float64       
 12  total_serious_injuries  70788 non-null  float6

In [69]:
# Fill the missing number of engine data with the best guess based on make and model
aviation_data_cleaned['number_of_engines'].fillna(value=aviation_data_cleaned['engine_number_map'], inplace=True)

In [70]:
# Check the number of missing numbers from the number of engines column
aviation_data_cleaned['number_of_engines'].isna().sum()

543

In [71]:
# Check the number of missing data in the dataset
aviation_data_cleaned.isna().sum()

event_date                   0
location                     0
airport_code                 0
airport_name                 0
injury_severity              0
aircraft_damage              0
make                         0
model                        0
number_of_engines          543
engine_type               2579
purpose_of_flight            0
total_fatal_injuries         0
total_serious_injuries       0
total_minor_injuries         0
total_uninjured              0
weather_condition            0
broad_phase_of_flight        0
engine_number_map          543
dtype: int64

In [72]:
# Check out the remaining missing data
aviation_data_cleaned[aviation_data_cleaned['number_of_engines'].isna()]

,event_date,location,airport_code,airport_name,injury_severity,aircraft_damage,make,model,number_of_engines,engine_type,purpose_of_flight,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight,engine_number_map
3870,1983-02-12,"LITTLEFIELD, TX",Q00,LITTLEFIELD,Non-Fatal,Substantial,Pratt-read,PRG-1,NaN,Unknown,Instructional,0.0,0.0,0.0,1.0,VMC,Cruise,NaN
4703,1983-05-17,"STOCKBRIDGE, MI",Unknown,Unknown,Non-Fatal,Substantial,Schaffer-provencher,UW,NaN,Unknown,Personal,0.0,1.0,0.0,2.0,VMC,Landing,NaN
4813,1983-05-28,"DECATUR, AL",Unknown,Unknown,Non-Fatal,Destroyed,Barnes,AX-6B,NaN,Unknown,Personal,0.0,1.0,1.0,0.0,VMC,Takeoff,NaN
4822,1983-05-28,"LOS LUNAS, NM",E98,MID VALLEY,Non-Fatal,Substantial,Glasflugel,H-301,NaN,Unknown,Personal,0.0,0.0,0.0,1.0,VMC,Takeoff,NaN
4957,1983-06-07,"RENO, NV",4SD,STEAD,Non-Fatal,Substantial,I.c.a. Brasov,IS-29D2,NaN,Unknown,Personal,0.0,0.0,0.0,2.0,VMC,Approach,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90102,2022-10-07,"Albuquerque, NM",Unknown,Unknown,Serious,Minor,AEROSTAR INTERNATIONAL,S53A,NaN,NaN,Personal,0.0,1.0,1.0,1.0,Unknown,Unknown,NaN
90205,2022-11-02,"New Orleans, LA",Unknown,Unknown,Serious,Unknown,AIRBUS,A319-132,NaN,NaN,Unknown,0.0,0.0,1.0,115.0,Unknown,Unknown,NaN
90259,2022-11-19,"Omaha, NE",Unknown,Unknown,Non-Fatal,Unknown,AIRBUS INDUSTRIE,A320-211,NaN,NaN,Unknown,0.0,0.0,0.0,153.0,Unknown,Unknown,NaN
90314,2022-12-08,"Newark, NJ",Unknown,Unknown,Serious,Unknown,BOEING,767-322,NaN,NaN,Unknown,0.0,3.0,1.0,175.0,Unknown,Unknown,NaN


In [73]:
# The remaining data can be dropped since it is a small amount
aviation_data_cleaned = aviation_data_cleaned[aviation_data_cleaned['number_of_engines'].notna()]

**Engine Type**
- We can take a similar map approach as the number of engines

In [74]:
# Create a map for the engine type based on the make and model
engine_type_dict = aviation_data_cleaned.groupby(['make', 'model', 'engine_type']).size().reset_index().rename(columns={0:'count'})
engine_type_dict

,make,model,engine_type,count
0,177MF LLC,PITTS MODEL 12,Reciprocating,1
1,2021FX3 LLC,CCX-2000,Reciprocating,1
2,AB SPORTINE AVIACIJA,LAK 17,Reciprocating,1
3,AERO ADVENTURE,ZEPHYR II,Reciprocating,1
4,AERO AT SP ZOO,AT-4 LSA,Reciprocating,1
...,...,...,...,...
9923,de Havilland,DHC-2,Reciprocating,4
9924,de Havilland,DHC-2 MK I,Reciprocating,1
9925,de Havilland,DHC-3,Turbo Prop,1
9926,de Havilland,DHC-6-200,Turbo Prop,1


In [75]:
# Check for Duplicates
engine_type_dict[['make', 'model', 'engine_type']].duplicated().sum()

0

In [76]:
# Create a new column to map the data to
engine_type_dict['make_model'] = tuple(zip(engine_type_dict['make'], engine_type_dict['model']))
engine_type_dict.head()

,make,model,engine_type,count,make_model
0,177MF LLC,PITTS MODEL 12,Reciprocating,1,"(177MF LLC, PITTS MODEL 12)"
1,2021FX3 LLC,CCX-2000,Reciprocating,1,"(2021FX3 LLC, CCX-2000)"
2,AB SPORTINE AVIACIJA,LAK 17,Reciprocating,1,"(AB SPORTINE AVIACIJA, LAK 17)"
3,AERO ADVENTURE,ZEPHYR II,Reciprocating,1,"(AERO ADVENTURE, ZEPHYR II)"
4,AERO AT SP ZOO,AT-4 LSA,Reciprocating,1,"(AERO AT SP ZOO, AT-4 LSA)"


In [77]:
# Make this data frame into a map
engine_type_map = dict(zip(engine_type_dict['make_model'], engine_type_dict['engine_type']))
list(engine_type_map.keys())[:5]

[('177MF LLC', 'PITTS MODEL 12'),
 ('2021FX3 LLC', 'CCX-2000'),
 ('AB SPORTINE AVIACIJA', 'LAK 17'),
 ('AERO ADVENTURE', 'ZEPHYR II'),
 ('AERO AT SP ZOO', 'AT-4 LSA')]

In [78]:
# Map the engine type to a column
aviation_data_cleaned['engine_type_map'] = tuple(zip(aviation_data_cleaned['make'], aviation_data_cleaned['model']))
aviation_data_cleaned['engine_type_map'] = aviation_data_cleaned['engine_type_map'].map(engine_type_map)
aviation_data_cleaned.head()


,event_date,location,airport_code,airport_name,injury_severity,aircraft_damage,make,model,number_of_engines,engine_type,purpose_of_flight,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight,engine_number_map,engine_type_map
7,1982-01-01,"PULLMAN, WA",Unknown,BLACKBURN AG STRIP,Non-Fatal,Substantial,Cessna,140,1.0,Reciprocating,Personal,0.0,0.0,0.0,2.0,VMC,Takeoff,1.0,Reciprocating
8,1982-01-01,"EAST HANOVER, NJ",N58,HANOVER,Non-Fatal,Substantial,Cessna,401B,2.0,Reciprocating,Business,0.0,0.0,0.0,2.0,IMC,Landing,2.0,Reciprocating
9,1982-01-01,"JACKSONVILLE, FL",JAX,JACKSONVILLE INTL,Non-Fatal,Substantial,North American,NAVION L-17B,1.0,Reciprocating,Personal,0.0,0.0,3.0,0.0,IMC,Cruise,1.0,Reciprocating
10,1982-01-01,"HOBBS, NM",Unknown,Unknown,Non-Fatal,Substantial,Piper,PA-28-161,1.0,Reciprocating,Personal,0.0,0.0,0.0,1.0,VMC,Approach,1.0,Unknown
11,1982-01-01,"TUSKEGEE, AL",Unknown,TUSKEGEE,Non-Fatal,Substantial,Beech,V35B,1.0,Reciprocating,Personal,0.0,0.0,0.0,1.0,VMC,Landing,1.0,Reciprocating


In [79]:
# Fill the missing number of engine data with the best guess based on make and model
aviation_data_cleaned['engine_type'].fillna(value=aviation_data_cleaned['engine_type_map'], inplace=True)

In [80]:
# Check out null counts
aviation_data_cleaned['engine_type'].isna().sum()

781

In [81]:
# We still have some null values so lets replace the rest with 'Unknown'
aviation_data_cleaned['engine_type'].fillna(value='Unknown', inplace=True)

In [82]:
# Check out null counts
aviation_data_cleaned['engine_type'].isna().sum()

0

In [83]:
# Drop Extra Columns
aviation_data_cleaned.drop(columns=['engine_number_map', 'engine_type_map'], inplace=True)

In [84]:
# Check out null counts for dataset
aviation_data_cleaned.isna().sum()

event_date                0
location                  0
airport_code              0
airport_name              0
injury_severity           0
aircraft_damage           0
make                      0
model                     0
number_of_engines         0
engine_type               0
purpose_of_flight         0
total_fatal_injuries      0
total_serious_injuries    0
total_minor_injuries      0
total_uninjured           0
weather_condition         0
broad_phase_of_flight     0
dtype: int64

In [85]:
# Check out our dataset after filling in NaNs
aviation_data_cleaned

,event_date,location,airport_code,airport_name,injury_severity,aircraft_damage,make,model,number_of_engines,engine_type,purpose_of_flight,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight
7,1982-01-01,"PULLMAN, WA",Unknown,BLACKBURN AG STRIP,Non-Fatal,Substantial,Cessna,140,1.0,Reciprocating,Personal,0.0,0.0,0.0,2.0,VMC,Takeoff
8,1982-01-01,"EAST HANOVER, NJ",N58,HANOVER,Non-Fatal,Substantial,Cessna,401B,2.0,Reciprocating,Business,0.0,0.0,0.0,2.0,IMC,Landing
9,1982-01-01,"JACKSONVILLE, FL",JAX,JACKSONVILLE INTL,Non-Fatal,Substantial,North American,NAVION L-17B,1.0,Reciprocating,Personal,0.0,0.0,3.0,0.0,IMC,Cruise
10,1982-01-01,"HOBBS, NM",Unknown,Unknown,Non-Fatal,Substantial,Piper,PA-28-161,1.0,Reciprocating,Personal,0.0,0.0,0.0,1.0,VMC,Approach
11,1982-01-01,"TUSKEGEE, AL",Unknown,TUSKEGEE,Non-Fatal,Substantial,Beech,V35B,1.0,Reciprocating,Personal,0.0,0.0,0.0,1.0,VMC,Landing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90340,2022-12-21,"Auburn Hills, MI",Unknown,Unknown,Minor,Unknown,CESSNA,172F,1.0,Reciprocating,Personal,0.0,1.0,0.0,0.0,Unknown,Unknown
90341,2022-12-21,"Reserve, LA",Unknown,Unknown,Minor,Unknown,GRUMMAN AMERICAN AVN. CORP.,AA-5B,1.0,Reciprocating,Instructional,0.0,1.0,0.0,1.0,Unknown,Unknown
90343,2022-12-26,"Annapolis, MD",Unknown,Unknown,Minor,Unknown,PIPER,PA-28-151,1.0,Reciprocating,Personal,0.0,1.0,0.0,0.0,Unknown,Unknown
90345,2022-12-26,"Payson, AZ",PAN,PAYSON,Non-Fatal,Substantial,AMERICAN CHAMPION AIRCRAFT,8GCBC,1.0,Reciprocating,Personal,0.0,0.0,0.0,1.0,VMC,Unknown


## Creating New Columns

The first column we will create will be the passenger count collumn. We will be assuming that:
- Passenger Count = [Total Injured] + [Total Uninjured]

In [87]:
aviation_data_cleaned['passenger_count'] = aviation_data_cleaned[['total_fatal_injuries','total_serious_injuries','total_minor_injuries','total_uninjured']].sum(axis=1)

In [88]:
aviation_data_cleaned

,event_date,location,airport_code,airport_name,injury_severity,aircraft_damage,make,model,number_of_engines,engine_type,purpose_of_flight,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight,passenger_count
7,1982-01-01,"PULLMAN, WA",Unknown,BLACKBURN AG STRIP,Non-Fatal,Substantial,Cessna,140,1.0,Reciprocating,Personal,0.0,0.0,0.0,2.0,VMC,Takeoff,2.0
8,1982-01-01,"EAST HANOVER, NJ",N58,HANOVER,Non-Fatal,Substantial,Cessna,401B,2.0,Reciprocating,Business,0.0,0.0,0.0,2.0,IMC,Landing,2.0
9,1982-01-01,"JACKSONVILLE, FL",JAX,JACKSONVILLE INTL,Non-Fatal,Substantial,North American,NAVION L-17B,1.0,Reciprocating,Personal,0.0,0.0,3.0,0.0,IMC,Cruise,3.0
10,1982-01-01,"HOBBS, NM",Unknown,Unknown,Non-Fatal,Substantial,Piper,PA-28-161,1.0,Reciprocating,Personal,0.0,0.0,0.0,1.0,VMC,Approach,1.0
11,1982-01-01,"TUSKEGEE, AL",Unknown,TUSKEGEE,Non-Fatal,Substantial,Beech,V35B,1.0,Reciprocating,Personal,0.0,0.0,0.0,1.0,VMC,Landing,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90340,2022-12-21,"Auburn Hills, MI",Unknown,Unknown,Minor,Unknown,CESSNA,172F,1.0,Reciprocating,Personal,0.0,1.0,0.0,0.0,Unknown,Unknown,1.0
90341,2022-12-21,"Reserve, LA",Unknown,Unknown,Minor,Unknown,GRUMMAN AMERICAN AVN. CORP.,AA-5B,1.0,Reciprocating,Instructional,0.0,1.0,0.0,1.0,Unknown,Unknown,2.0
90343,2022-12-26,"Annapolis, MD",Unknown,Unknown,Minor,Unknown,PIPER,PA-28-151,1.0,Reciprocating,Personal,0.0,1.0,0.0,0.0,Unknown,Unknown,1.0
90345,2022-12-26,"Payson, AZ",PAN,PAYSON,Non-Fatal,Substantial,AMERICAN CHAMPION AIRCRAFT,8GCBC,1.0,Reciprocating,Personal,0.0,0.0,0.0,1.0,VMC,Unknown,1.0


## Column Cleaning

This section is for cleaning columns.
- Clean make to make sure there aren't duplicate makes
- Separate out City and State

### Makes

In [89]:
# Start by checking out the different makes
aviation_data_cleaned['make'].unique()

array(['Cessna', 'North American', 'Piper', ..., 'PHANTOM',
       'JAMES R DERNOVSEK', 'ORLICAN S R O'], dtype=object)

In [90]:
# Find the current number of unique makes
len(aviation_data_cleaned['make'].unique())

2050

In [91]:
# Change names to be capitalized and check the length
cleaned_makes = aviation_data_cleaned['make'].str.title().unique()
print(cleaned_makes)
len(cleaned_makes)

['Cessna' 'North American' 'Piper' ... 'Phantom' 'James R Dernovsek'
 'Orlican S R O']


1683

In [92]:
# That looks like we cleaned a bunch of duplicates, let's assign the column and check
aviation_data_cleaned['make'] = aviation_data_cleaned['make'].str.title()

In [93]:
# Check the length
len(aviation_data_cleaned['make'].unique())

1683

### Clean the Top 5 makes specifically

In [94]:
aviation_data_cleaned.shape

(70245, 18)

In [95]:
## Create a list of top brands and a map
top_makes_list = ['Beech', 'Cessna', 'Cirrus', 'Piper', 'Diamond',
                  'Lockheed', 'Airbus', 'Grumman', 'Raytheon', 'Boeing']
top_makes_map = {}

for make in top_makes_list:
    beech_dict = aviation_data_cleaned[aviation_data_cleaned['make']
                                       .str.contains(make)]['make'].value_counts().to_frame()
    beech_dict['make'] = make
    top_makes_map.update(dict(beech_dict['make']))
# Check out the top makes map
top_makes_map

{'Beech': 'Beech',
 'Beechcraft': 'Beech',
 'Hawker Beechcraft Corp': 'Beech',
 'Hawker Beechcraft': 'Beech',
 'Hawker Beechcraft Corporation': 'Beech',
 'Hawker Beechcraft Corp.': 'Beech',
 'Hawker-Beechcraft': 'Beech',
 'Hawker-Beechcraft Corporation': 'Beech',
 'Beechcraft Corporation': 'Beech',
 'Hawker Beech': 'Beech',
 'Cessna': 'Cessna',
 'Cessna Aircraft Co': 'Cessna',
 'Cessna Ector': 'Cessna',
 'Cessna/Air Repair Inc': 'Cessna',
 'Cessna Wren': 'Cessna',
 'Cessna Aircraft Co.': 'Cessna',
 'Cessna Robertson': 'Cessna',
 'Cessna Soloy': 'Cessna',
 'Cessna/Weaver': 'Cessna',
 'Cessna Aircraft': 'Cessna',
 'Cirrus Design Corp': 'Cirrus',
 'Cirrus': 'Cirrus',
 'Cirrus Design Corp.': 'Cirrus',
 'Cirrus Design Corporation': 'Cirrus',
 'Cirrus Design': 'Cirrus',
 'Piper': 'Piper',
 'Piper Aircraft Inc': 'Piper',
 'Piper/Cub Crafters': 'Piper',
 'New Piper Aircraft Inc': 'Piper',
 'Piper Aircraft': 'Piper',
 'New Piper': 'Piper',
 'Piper Aerostar': 'Piper',
 "Piper/Wally'S Flyers Inc"

In [96]:
# Apply the Top Makes Map to the data
top_makes_filter = aviation_data_cleaned['make'].isin(top_makes_list)
aviation_data_cleaned[top_makes_filter]['make'] = aviation_data_cleaned['make'].map(top_makes_map)

<ipython-input-96-9edcf83bf972>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aviation_data_cleaned[top_makes_filter]['make'] = aviation_data_cleaned['make'].map(top_makes_map)


In [97]:
aviation_data_cleaned.shape

(70245, 18)

In [98]:
aviation_data_cleaned['top_make'] = aviation_data_cleaned['make'].isin(top_makes_list)

In [99]:
aviation_data_cleaned['top_make'].value_counts(normalize=True)

True     0.656516
False    0.343484
Name: top_make, dtype: float64

In [100]:
len(aviation_data_cleaned[aviation_data_cleaned['top_make']]['make'].value_counts())

10

In [132]:
aviation_data_cleaned[aviation_data_cleaned['top_make']].groupby('make').count()

,event_date,location,airport_code,airport_name,injury_severity,aircraft_damage,model,number_of_engines,engine_type,purpose_of_flight,...,weather_condition,broad_phase_of_flight,passenger_count,top_make,use_category,top_model,city,state,fatality_rate,percent_uninjured
make,,,,,,,,,,,,,,,,,,,,,
Airbus,69,69,69,69,69,69,69,69,69,69,...,69,69,69,69,69,69,69,69,69,69
Beech,4838,4838,4838,4838,4838,4838,4838,4838,4838,4838,...,4838,4838,4838,4838,4838,4838,4838,4838,4838,4838
Boeing,868,868,868,868,868,868,868,868,868,868,...,868,868,868,868,868,868,868,868,868,868
Cessna,25157,25157,25157,25157,25157,25157,25157,25157,25157,25157,...,25157,25157,25157,25157,25157,25157,25157,25157,25157,25157
Cirrus,96,96,96,96,96,96,96,96,96,96,...,96,96,96,96,96,96,96,96,96,96
Diamond,35,35,35,35,35,35,35,35,35,35,...,35,35,35,35,35,35,35,35,35,35
Grumman,1110,1110,1110,1110,1110,1110,1110,1110,1110,1110,...,1110,1110,1110,1110,1110,1110,1110,1110,1110,1110
Lockheed,66,66,66,66,66,66,66,66,66,66,...,66,66,66,66,66,66,66,66,66,66
Piper,13830,13830,13830,13830,13830,13830,13830,13830,13830,13830,...,13830,13830,13830,13830,13830,13830,13830,13830,13830,13830


### Determine Commercial or Private

In [101]:
def private_or_commercial(make):
    private = ['Beech', 'Cessna', 'Cirrus', 'Piper', 'Diamond']
    commercial = ['Lockheed', 'Airbus', 'Grumman', 'Raytheon', 'Boeing']
    if make in private:
        return 'Private Enterprise'
    elif make in commercial:
        return 'Commercial'
    else:
        return 'Unknown'

In [102]:
# Create a column to designate commercial or private
aviation_data_cleaned['use_category'] = aviation_data_cleaned['make'].apply(private_or_commercial)
aviation_data_cleaned['use_category'].value_counts()

Private Enterprise    43986
Unknown               24128
Commercial             2131
Name: use_category, dtype: int64

### Top 3 Models per make

In [103]:
# Remove exccess white space from data and remove "-"
aviation_data_cleaned['model'] = aviation_data_cleaned['model'].str.strip().str.replace('-','').str.replace(' ','')
aviation_data_cleaned['model'].head()

7            140
8           401B
9     NAVIONL17B
10       PA28161
11          V35B
Name: model, dtype: object

In [104]:
def model_cleaning_function(model):
    model = str(model)
    if model.startswith('1') or model.startswith('2'):
         model = model[:3]
    elif any(map(model.startswith, ['PA','A','DA','G','SR','C','L'])):
        model = model[:4]
    elif model.startswith('HAWKER'):
         model = 'HAWKER'
    else: 
        model = model
    return model

In [105]:
aviation_data_cleaned['model'] = aviation_data_cleaned['model'].apply(model_cleaning_function)

In [106]:
top_commercial_models_list = ['A330', '747', '777', '727', 'CRJ', 'A320', '747', '737', 'DC3']

In [107]:
only_private = aviation_data_cleaned['use_category'] == 'Private Enterprise'
top_models_list = aviation_data_cleaned[top_makes_filter & only_private].groupby(['make','model']).count().reset_index()
top_models_list = list(top_models_list.sort_values('event_date',ascending=False).groupby('make').head(3)['model'])
top_models_list.extend(top_commercial_models_list)
top_models_list

['172',
 'PA28',
 '150',
 '182',
 'PA18',
 'PA32',
 'A36',
 'C23',
 '58',
 'SR22',
 'DA20',
 'SR20',
 'DA40',
 'DA42',
 'A330',
 '747',
 '777',
 '727',
 'CRJ',
 'A320',
 '747',
 '737',
 'DC3']

**Top Commercial Models**
- Airbus A330
- Boeing 747
- Boeing 777
- Boeing 727
- CRJ Series
- Airbus A320
- Boeing 747
- Embraer E-Jet Family
- Boeing 737
- Douglas DC-3

In [108]:
def commercial_model_cleaning_function(model):
    top_commercial_models_list = ['A330', '747', '777', '727', 'CRJ', 'A320', '747', '737', 'DC3']
    model = str(model)
    for top_model in top_commercial_models_list:
        if model.startswith(top_model):
            return top_model
            break
        else:
            return model

In [109]:
aviation_data_cleaned['model'] = aviation_data_cleaned['model'].apply(commercial_model_cleaning_function)

In [110]:
aviation_data_cleaned[aviation_data_cleaned['model'].isin(top_commercial_models_list)]['model'].count()

108

In [111]:
aviation_data_cleaned['top_model'] = aviation_data_cleaned['model'].isin(top_models_list)

In [112]:
aviation_data_cleaned['top_model'].value_counts(normalize=True)

False    0.701274
True     0.298726
Name: top_model, dtype: float64

### Location to City & State

In [113]:
# Add new city and state columns
aviation_data_cleaned[['city', 'state']] = aviation_data_cleaned['location'].str.rsplit(', ', 1, expand = True)

In [114]:
aviation_data_cleaned['city'] = aviation_data_cleaned['city'].str.capitalize()

In [115]:
aviation_data_cleaned[['city', 'state']].head()

,city,state
7,Pullman,WA
8,East hanover,NJ
9,Jacksonville,FL
10,Hobbs,NM
11,Tuskegee,AL


In [116]:
# Check for missing data
aviation_data_cleaned[['city', 'state']].isna().sum()

city      0
state    51
dtype: int64

In [117]:
# There are only a few so we can remove them
aviation_data_cleaned = aviation_data_cleaned[aviation_data_cleaned['state'].notna()]

In [118]:
# Confirm the missing data is gone
aviation_data_cleaned[['city', 'state']].isna().sum()

city     0
state    0
dtype: int64

In [119]:
# Confirm the missing data is gone
aviation_data_cleaned.isna().sum()

event_date                0
location                  0
airport_code              0
airport_name              0
injury_severity           0
aircraft_damage           0
make                      0
model                     0
number_of_engines         0
engine_type               0
purpose_of_flight         0
total_fatal_injuries      0
total_serious_injuries    0
total_minor_injuries      0
total_uninjured           0
weather_condition         0
broad_phase_of_flight     0
passenger_count           0
top_make                  0
use_category              0
top_model                 0
city                      0
state                     0
dtype: int64

### Weather conditions

In [120]:
aviation_data_cleaned['weather_condition'].value_counts()

VMC        63918
IMC         5231
Unknown      484
UNK          473
Unk           88
Name: weather_condition, dtype: int64

In [121]:
# Update weather conditions names
weather_map = { 'VMC' : 'Visual Meteorological Conditions',
                 'IMC' : 'Instrument Meteorological Conditions',
                 'Unknown' : 'Unknown Meteorological Conditions',
                 'UNK' : 'Unknown Meteorological Conditions',
                 'Unk' : 'Unknown Meteorological Conditions'
}

aviation_data_cleaned['weather_condition'] = aviation_data_cleaned['weather_condition'].map(weather_map)

<ipython-input-121-41ea5d073c37>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aviation_data_cleaned['weather_condition'] = aviation_data_cleaned['weather_condition'].map(weather_map)


### Injury Severity

In [122]:
# Check out injury severity
aviation_data_cleaned['injury_severity'].value_counts()

Non-Fatal      57453
Fatal(1)        4556
Fatal(2)        2989
Fatal           2851
Fatal(3)         955
Fatal(4)         663
Minor            178
Fatal(5)         166
Serious          128
Fatal(6)         104
Fatal(7)          30
Fatal(8)          26
Fatal(10)         15
Unavailable       14
Fatal(9)           8
Fatal(14)          5
Fatal(12)          5
Fatal(13)          3
Fatal(11)          3
Fatal(18)          3
Fatal(23)          2
Fatal(20)          2
Fatal(17)          2
Fatal(25)          2
Fatal(228)         1
Fatal(21)          1
Fatal(15)          1
Fatal(19)          1
Fatal(88)          1
Fatal(49)          1
Fatal(78)          1
Fatal(153)         1
Fatal(44)          1
Fatal(31)          1
Fatal(16)          1
Fatal(92)          1
Fatal(37)          1
Fatal(28)          1
Fatal(135)         1
Fatal(64)          1
Fatal(110)         1
Fatal(230)         1
Fatal(73)          1
Fatal(111)         1
Fatal(68)          1
Fatal(132)         1
Fatal(265)         1
Fatal(34)    

In [123]:
# Clean up the injury severity
aviation_data_cleaned['injury_severity'] = aviation_data_cleaned['injury_severity'].str.split('(', 1, expand = True)

<ipython-input-123-8a01635113c5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aviation_data_cleaned['injury_severity'] = aviation_data_cleaned['injury_severity'].str.split('(', 1, expand = True)


In [124]:
aviation_data_cleaned['injury_severity'].head()

7     Non-Fatal
8     Non-Fatal
9     Non-Fatal
10    Non-Fatal
11    Non-Fatal
Name: injury_severity, dtype: object

## Create key metrics
- Fatality rate
- % uninjured

In [125]:
# Create a column call fatality rate to figure out the deadliness of the accident
aviation_data_cleaned['fatality_rate'] = aviation_data_cleaned['total_fatal_injuries']/aviation_data_cleaned['passenger_count']
# Create a column call fatality rate to figure out the deadliness of the accident
aviation_data_cleaned['percent_uninjured'] = aviation_data_cleaned['total_uninjured']/aviation_data_cleaned['passenger_count']

<ipython-input-125-f7f133f62319>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aviation_data_cleaned['fatality_rate'] = aviation_data_cleaned['total_fatal_injuries']/aviation_data_cleaned['passenger_count']
<ipython-input-125-f7f133f62319>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aviation_data_cleaned['percent_uninjured'] = aviation_data_cleaned['total_uninjured']/aviation_data_cleaned['passenger_count']


In [131]:
aviation_data_cleaned.fillna(0, inplace=True)

/Users/willbennett/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4317: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


## Conclusion
We now have a cleaned up data set with mostly non-null values and have focused on the columns we need

In [126]:
aviation_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70194 entries, 7 to 90347
Data columns (total 25 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   event_date              70194 non-null  datetime64[ns]
 1   location                70194 non-null  object        
 2   airport_code            70194 non-null  object        
 3   airport_name            70194 non-null  object        
 4   injury_severity         70194 non-null  object        
 5   aircraft_damage         70194 non-null  object        
 6   make                    70194 non-null  object        
 7   model                   70194 non-null  object        
 8   number_of_engines       70194 non-null  float64       
 9   engine_type             70194 non-null  object        
 10  purpose_of_flight       70194 non-null  object        
 11  total_fatal_injuries    70194 non-null  float64       
 12  total_serious_injuries  70194 non-null  float6